In [26]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import torch

from transformers import AutoTokenizer, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import DataLoader

In [27]:
path = "../trainedModels"
model_name = "google/electra-base-discriminator"

In [28]:
# Load dataset
dataset = load_dataset('csv', data_files={
    'train': '../transformers_data/train.csv',
    'validation': '../transformers_data/validation.csv',
    'test': '../transformers_data/test.csv'
})

# Define label mappings
label_mapping = {
    "Effective": 0,
    "Adequate": 1,
    "Ineffective": 2
}

label_mapping_types = {
    "Claim": 0,
    "Concluding Statement": 1,
    "Counterclaim": 2,
    "Evidence": 3,
    "Lead": 4,
    "Position": 5,
    "Rebuttal": 6
}

def encode_labels(example):
    example['labels'] = label_mapping[example['discourse_effectiveness']]
    return example

def encode_types(example):
    example['types'] = label_mapping_types[example['discourse_type']]
    return example

dataset = dataset.map(encode_labels)
dataset = dataset.map(encode_types)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    texts = [text if isinstance(text, str) else "" for text in examples['discourse_text']]
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=256
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns([
    'discourse_id', 'essay_id', 'discourse_text', 
    'discourse_effectiveness', 'discourse_type'
])
tokenized_datasets.set_format('torch')

# Initialize model
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=num_labels,
    problem_type="single_label_classification"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, 
        predictions, 
        average='weighted'
    )
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Modified training arguments specifically for MPS
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",
    eval_steps=100,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.1,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    save_strategy='steps',
    save_steps=100,
    save_total_limit=2,
    warmup_ratio=0.1,
    logging_steps=50,
    gradient_accumulation_steps=2,
    # Disable all precision options
    fp16=False,
    fp16_full_eval=False,
    bf16=False,
    # Disable CUDA specific options
    local_rank=-1,
    dataloader_pin_memory=False,
    use_cpu=False,
    # No distributed training
    ddp_find_unused_parameters=False,
    no_cuda=True
)

# Initialize trainer with no specific accelerator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate on test set
results = trainer.evaluate(tokenized_datasets['test'])
print(results)

Map: 100%|██████████| 3677/3677 [00:00<00:00, 24417.49 examples/s]
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/training_args.py:1560: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Save model and tokenizer
model_save_path = f'{path}/{model_name.split("/")[-1]}'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Save training history
with open(f'{model_save_path}/training_history.pkl', 'wb') as f:
    pickle.dump(trainer.state.log_history, f)

# Make and save predictions
predictions = trainer.predict(tokenized_datasets['test'])
with open(f'{model_save_path}/predictions.pkl', 'wb') as f:
    pickle.dump(predictions, f)